In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm.evaluation import auc_score
import implicit

In [ ]:
#Creating a Collaborative filtering model

In [ ]:
description = pd.read_csv('description.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
description.head()

In [ ]:
description.info()

In [ ]:
ratings.tail()

In [ ]:
description['type'].value_counts()

In [ ]:
description = description[description['type']=='TV']

In [ ]:
data = ratings.merge(description, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
data.rename(columns = {'rating_user':'user_rating'}, inplace = True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data=data[data['user_id']<=15000]

In [ ]:
data.shape

In [ ]:
pivot = data.pivot_table(index=['user_id'], columns=[
                         'name'], values='user_rating', fill_value=0).reset_index(drop=True)
print(pivot.shape)

In [ ]:
pivot=pivot.reindex(sorted(pivot.columns), axis=1)

In [ ]:
pivot[pivot != 0] = 1
pivot=pivot.loc[~(pivot==0).all(axis=1)]

In [ ]:
pivot_sparse = csr_matrix(pivot.values)

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.01, iterations=3)
model.fit(pivot_sparse.T)

In [ ]:
model.item_factors

In [ ]:
model.user_factors

In [ ]:
pivot.T[pivot.T.iloc[:, 14053]!=0][14053]

In [ ]:
model.recommend(userid=14053, user_items=pivot_sparse,
                N=3, filter_already_liked_items=True)

In [ ]:
model.explain(userid=14053, user_items=pivot_sparse, itemid=610)[1]

In [ ]:
print('Recommended show: ' + pivot.columns[610])
print('______________________________________________')
print('The show was recommended because these shows are usually watched together with it by users : ',
      pivot.columns[609], '+', pivot.columns[615], '+', pivot.columns[1467])

In [ ]:
# find related items
related = model.similar_items(itemid=610, N=3)
print([pivot.columns[x[0]] for x in related])

In [ ]:
#Developing a hybrid Recommendation system

In [ ]:
description = description[['name', 'genre', 'episodes', 'rating']]
# description['episodes'] = 1 for the sake of our experiment
description.loc[description['episodes']=='Unknown', 'episodes'] = 1
description['episodes'] = description['episodes'].astype(int)

In [ ]:
description['episodes'].hist(bins=5)

In [ ]:
description['rating'].hist()

In [ ]:
description = pd.get_dummies(description, columns=['genre'])

In [ ]:
description.head()

In [ ]:
description_sparse = csr_matrix(description.values)

In [ ]:
description_sparse.toarray()
description_sparse.shape